# RoBERTa for Office and General: 

In [1]:
# install transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [1]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
tf.random.set_seed(1234)

In [2]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [24]:
# load csvs: 
# train_df = pd.read_csv('train_df_processed.csv')
# test_df =pd.read_csv('test_df_processed.csv')
import pandas as pd
import os
train_path = '/home/ec2-user/SageMaker/amazon_thesis_detectron/general_training_data'
zero_star = pd.read_csv(os.path.join(train_path, 'zero_star_general.csv'))
zero_star.columns

Index(['review_id', 'full_review', 'zero_star', 'zero_star_ohe'], dtype='object')

In [2]:
# load csvs: 
# train_df = pd.read_csv('train_df_processed.csv')
# test_df =pd.read_csv('test_df_processed.csv')
import pandas as pd
import os
train_path = '/home/ec2-user/SageMaker/amazon_thesis_detectron/general_training_data'
zero_star = pd.read_csv(os.path.join(train_path, 'zero_star_general.csv'))
zero_star.columns

Index(['review_id', 'full_review', 'zero_star', 'zero_star_ohe'], dtype='object')

In [3]:
zero_star['sentence_length']= zero_star['full_review'].apply(lambda x: len(x.split()))
def create_sentence_list(df, label): 
    df_1 = df[df[label]==1]
    return df_1['sentence_length'].tolist()
wrong = create_sentence_list(zero_star, 'zero_star_ohe')
# Calculate the average
w = sum(wrong) / len(wrong)

print("Average:", w)

Average: 81.91316931982634


In [25]:
zero_star.zero_star_ohe.value_counts()

0    9848
1    1382
Name: zero_star_ohe, dtype: int64

In [26]:
zero_star= zero_star[['zero_star_ohe', 'full_review']]

In [27]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(zero_star, test_size= 0.3, random_state=42, stratify=zero_star['zero_star_ohe'])

In [28]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.2, random_state=42, stratify=train['zero_star_ohe'])

In [29]:
val_df.zero_star_ohe.value_counts()

0    1380
1     193
Name: zero_star_ohe, dtype: int64

In [30]:
val_df[val_df.zero_star_ohe==1].full_review.iloc[0]

'After 4 months of use my 7 year pen stopped writing. I give it one star for the sustainability idea but it deserves 0 stars for execution. How do I get my money back?'

In [31]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [32]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'zero_star_ohe',8 )
train_dataset =train_tensor.tensor_data() 

6288it [00:07, 811.63it/s]


In [33]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'zero_star_ohe', 8)
    val_dataset =val_tensor.tensor_data() 

1573it [00:02, 703.75it/s]


In [23]:
# Defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # Pooled output
    
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer1')(bert_embds)
    intermediate_layer = tf.keras.layers.Dropout(0.4, name='dropout1')(intermediate_layer)
    
    intermediate_layer = tf.keras.layers.Dense(256, activation='relu', name='intermediate_layer2')(intermediate_layer)
    intermediate_layer = tf.keras.layers.Dropout(0.3, name='dropout2')(intermediate_layer)
    
    intermediate_layer = tf.keras.layers.Dense(128, activation='relu', name='intermediate_layer3')(intermediate_layer)
    intermediate_layer = tf.keras.layers.Dropout(0.2, name='dropout3')(intermediate_layer)
    
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # Sigmoid activation for binary classification
    
    model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
786/786 [==============================] - 400s 466ms/step - loss: 0.2121 - accuracy: 0.9346 - val_loss: 0.0829 - val_accuracy: 0.9790
Epoch 2/100
786/786 [==============================] - 362s 460ms/step - loss: 0.0795 - accuracy: 0.9796 - val_loss: 0.0809 - val_accuracy: 0.9777
Epoch 3/100
134/786 [====>.........................] - ETA: 4:38 - loss: 0.0452 - accuracy: 0.9907

In [24]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
786/786 [==============================] - 392s 457ms/step - loss: 0.1669 - accuracy: 0.9448 - val_loss: 0.0758 - val_accuracy: 0.9802
Epoch 2/100
786/786 [==============================] - 352s 448ms/step - loss: 0.0638 - accuracy: 0.9846 - val_loss: 0.0787 - val_accuracy: 0.9815
Epoch 3/100
786/786 [==============================] - 350s 446ms/step - loss: 0.0544 - accuracy: 0.9855 - val_loss: 0.0834 - val_accuracy: 0.9790
Epoch 4/100
786/786 [==============================] - 351s 446ms/step - loss: 0.0364 - accuracy: 0.9908 - val_loss: 0.0755 - val_accuracy: 0.9821
Epoch 5/100
786/786 [==============================] - 350s 446ms/step - loss: 0.0308 - accuracy: 0.9919 - val_loss: 0.0775 - val_accuracy: 0.9809
Epoch 6/100
786/786 [==============================] - 350s 445ms/step - loss: 0.0253 - accuracy: 0.9924 - val_loss: 0.1025 - val_accuracy: 0.9713
Epoch 7/100
786/786 [==============================] - 352s 448ms/step - loss: 0.0156 - accuracy: 0.9955 - val_loss: 0

In [25]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [27]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


...

1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 58ms/step


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['zero_star_ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1140
           1       0.89      0.96      0.92       195

    accuracy                           0.98      1335
   macro avg       0.94      0.97      0.96      1335
weighted avg       0.98      0.98      0.98      1335



In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['zero_star_ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2954
           1       0.92      0.91      0.92       415

    accuracy                           0.98      3369
   macro avg       0.96      0.95      0.95      3369
weighted avg       0.98      0.98      0.98      3369



In [29]:
model.save('zero_star_general_fnl')

INFO:tensorflow:Assets written to: zero_star_general_fnl/assets


INFO:tensorflow:Assets written to: zero_star_general_fnl/assets


In [39]:
test_df[test_df['result']==1].iloc[1].full_review

'This is the most overpriced piece of junk I have ever purchased online. It is the equivalent of a piece of cardboard with a gray spray painted surface. I cannot imagine even selling this item. I feel I am cheating the public by giving this even one star.'

## Conclusion: 

The RoBERTa model was able to achieve a accuracy of ~95% on the testing dataset. It can also be seen that there is a lot more parameters involved in comparison to BERT embeddings. The intresting feature of RoBERTa is that it can take a max length of upto 1024, however, given the sentence length distribution for the task at hand, max length remained at 512. Like BERT, it is able to predict significantly in comparison to other models but remains extremely computationally expensive for the task at hand. I utilised the pooled output from the roberta model for the task at hand too. 

Accuracy achieved - 95%. 